In [1]:
print("qwerty")

qwerty


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone 
from langchain_pinecone import PineconeVectorStore
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.document_loaders import CSVLoader

c:\Users\ayush\.conda\envs\chatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [110]:
from langchain.llms import OpenAI

In [128]:
from langchain.chat_models import ChatOpenAI

In [74]:
from langchain_openai import OpenAIEmbeddings

In [3]:
import os

In [77]:
os.environ["OPENAI_API_KEY"] = 

In [80]:
os.environ['PINECONE_API_KEY'] = 
os.environ['PINECONE_INDEX_NAME'] = 


In [5]:
from pathlib import Path

In [82]:
PINECONE_API_KEY = "5c8d0e8e-4a48-4b6f-953f-e8cfad1c3f4e"
PINECONE_INDEX = "open-ai"
INDEX_DIMENSIONS = 1536

In [60]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [61]:
from langchain_community.document_loaders import TextLoader



In [62]:
loader = TextLoader("data\output.txt")
txt_data= loader.load()

In [63]:
text_chunks = text_split(txt_data)
print("Number of chunks:", len(text_chunks))

Number of chunks: 211


In [64]:
text_chunks

[Document(page_content='Internet users in 1960 was Data not available in Brazil by International Telecommunication Union.', metadata={'source': 'data\\output.txt'}),
 Document(page_content='Union. Poverty in 1960 was Data not available  in Brazil by World Bank, Poverty and Inequality', metadata={'source': 'data\\output.txt'}),
 Document(page_content='and Inequality Platform. Unemployment in 1960 was Data not available  in Brazil by International', metadata={'source': 'data\\output.txt'}),
 Document(page_content='by International Labour Organization. Internet users in 1961 was Data not available in Brazil by', metadata={'source': 'data\\output.txt'}),
 Document(page_content='in Brazil by International Telecommunication Union. Poverty in 1961 was Data not available  in', metadata={'source': 'data\\output.txt'}),
 Document(page_content='not available  in Brazil by World Bank, Poverty and Inequality Platform. Unemployment in 1961 was', metadata={'source': 'data\\output.txt'}),
 Document(pa

In [75]:
def download_hugging_face_embeddings():
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    return embeddings

In [78]:
embeddings = download_hugging_face_embeddings()

In [79]:
query_result = embeddings.embed_query("Sample")
print("Length", len(query_result))

Length 1536


In [83]:
for t in text_chunks:
    print(t.page_content)

Internet users in 1960 was Data not available in Brazil by International Telecommunication Union.
Union. Poverty in 1960 was Data not available  in Brazil by World Bank, Poverty and Inequality
and Inequality Platform. Unemployment in 1960 was Data not available  in Brazil by International
by International Labour Organization. Internet users in 1961 was Data not available in Brazil by
in Brazil by International Telecommunication Union. Poverty in 1961 was Data not available  in
not available  in Brazil by World Bank, Poverty and Inequality Platform. Unemployment in 1961 was
in 1961 was Data not available  in Brazil by International Labour Organization. Internet users in
Internet users in 1962 was Data not available in Brazil by International Telecommunication Union.
Union. Poverty in 1962 was Data not available  in Brazil by World Bank, Poverty and Inequality
and Inequality Platform. Unemployment in 1962 was Data not available  in Brazil by International
by International Labour Organiza

In [84]:
index_name= PINECONE_INDEX

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=PineconeVectorStore.from_texts([t.page_content for t in text_chunks], index_name=index_name,embedding=embeddings)

In [104]:

query = "number of internet users in 2010"

docs=docsearch.similarity_search(query, k=5)

print("Result", docs)

Result [Document(page_content='Internet users in 2010 was 40.65 in Brazil by International Telecommunication Union. Poverty in'), Document(page_content='in 2015 was 8.54  in Brazil by International Labour Organization. Internet users in 2016 was'), Document(page_content='Internet users in 2000 was 2.870685159 in Brazil by International Telecommunication Union. Poverty'), Document(page_content='Internet users in 2006 was 28.17838018 in Brazil by International Telecommunication Union. Poverty'), Document(page_content='Internet users in 2005 was 21.02274725 in Brazil by International Telecommunication Union. Poverty')]


In [143]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If the question is not relevant to the context ask the user to ask a relevant question.
Do not answer any irrelevant questions.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [144]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [145]:
openai_llm = OpenAI(model="text-davinci-002")

In [146]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.3})

In [147]:
llm4 = ChatOpenAI(
    openai_api_key ##Insert your Key,
    model_name='gpt-4',
    temperature=0.0
)

In [148]:
# from langchain.schema.retriever import BaseRetriever, Document
# from typing import List

# class SimpleRetriever(BaseRetriever):
#     def __init__(self, vector_store, k=5):
#         self.vector_store = vector_store
#         self.k = k

#     def _get_relevant_documents(self, query: str) -> List[Document]:
#         """Retrieve documents based on embeddings similarity, limited to the top k results."""
#         # Assuming `vector_store` has a method `similarity_search` which takes a query and returns documents
#         results = self.vector_store.similarity_search(query, k=self.k)
#         return results

#     async def _aget_relevant_documents(self, query: str) -> List[Document]:
#         """Asynchronous retrieval of documents."""
#         return await self.vector_store.similarity_search(query, k=self.k)


In [149]:
from typing import List, Optional, Any  # Adding 'Any' to the import
from langchain.schema import Document, BaseRetriever
from pydantic import BaseModel, Field

In [150]:
# from pydantic import BaseModel, Field
# from typing import List, Optional
# from langchain.schema import Document, BaseRetriever

# class SimpleRetriever(BaseRetriever):
#     vector_store: Any  # Specify the correct type if known, else use `Any`
#     k: int = Field(default=5, description="The number of documents to retrieve")

#     def _get_relevant_documents(self, query: str) -> List[Document]:
#         """Retrieve documents based on embeddings similarity, limited to the top k results."""
#         # Simulate fetching documents based on a query
#         # This is where you'd normally interact with the vector store
#         return self.vector_store.get_relevant_documents(query, k=self.k)

#     async def _aget_relevant_documents(self, query: str) -> List[Document]:
#         """Asynchronously retrieve documents."""
#         return await self.vector_store.get_relevant_documents(query, k=self.k)


In [151]:
from langchain.schema import Document, BaseRetriever
from typing import List, Any
from pydantic import BaseModel, Field

class SimpleRetriever(BaseRetriever):
    vector_store: Any  # Specify the correct type if known, else use `Any`
    k: int = Field(default=5, description="The number of documents to retrieve")

    def _get_relevant_documents(self, query: str) -> List[Document]:
        """Retrieve documents based on embeddings similarity, limited to the top k results using cosine similarity."""
        return self.vector_store.similarity_search(query=query, k=self.k)

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        """Asynchronously retrieve documents using cosine similarity."""
        return await self.vector_store.similarity_search(query=query, k=self.k)


In [152]:
# from langchain.schema import Document, BaseRetriever
# from typing import List, Any
# from pydantic import BaseModel, Field

# class SimpleRetriever(BaseRetriever):
#     vector_store: Any  # Specify the correct type if known, else use `Any`
#     k: int = Field(default=5, description="The number of documents to retrieve")

#     def _get_relevant_documents(self, query: str) -> List[Document]:
#         """Retrieve documents based on embeddings similarity, limited to the top k results using cosine similarity."""
#         return self.vector_store.search(query=query, k=self.k, search_type='similarity')

#     async def _aget_relevant_documents(self, query: str) -> List[Document]:
#         """Asynchronously retrieve documents using cosine similarity."""
#         return await self.vector_store.search(query=query, k=self.k, search_type='similarity')


In [153]:
# Assuming docsearch is already set up as an instance of PineconeVectorStore
simple_retriever = SimpleRetriever(vector_store=docsearch, k=5)


In [154]:
doc = simple_retriever._get_relevant_documents("number of internet users in 2020")

In [155]:
doc

[Document(page_content='Internet users in 2020 was 81.34269353 in Brazil by International Telecommunication Union. Poverty'),
 Document(page_content='Internet users in 2022 was 80.52775094 in Brazil by International Telecommunication Union. Poverty'),
 Document(page_content='Internet users in 2021 was 80.68989314 in Brazil by International Telecommunication Union. Poverty'),
 Document(page_content='Internet users in 2019 was 73.91243991 in Brazil by International Telecommunication Union. Poverty'),
 Document(page_content='Internet users in 2018 was 70.43428254 in Brazil by International Telecommunication Union. Poverty')]

In [156]:
qa = RetrievalQA.from_chain_type(  
    llm=llm4,  
    chain_type="stuff",  
    retriever=simple_retriever,
    chain_type_kwargs=chain_type_kwargs 
)  

In [163]:

user_input=input(f"Input Prompt:")


In [164]:
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  This is not a relevant question related to the context provided.
